In [3]:
#!pip install gensim
#!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 2.7 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149872 sha256=d2c2ebd9efa30a2e0dc34862563534de98bdd7a1c497bfa2716cc2bc99f2a82b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [4]:
import numpy as np
import pandas as pd

import gensim

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_json('/content/drive/MyDrive/Datasets/Sports_and_Outdoors_5.json', lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIXZKN4ACSKI,1881509818,David Briner,"[0, 0]",This came in on time and I am veru happy with ...,5,Woks very good,1390694400,"01 26, 2014"
1,A1L5P841VIO02V,1881509818,Jason A. Kramer,"[1, 1]",I had a factory Glock tool that I was using fo...,5,Works as well as the factory tool,1328140800,"02 2, 2012"
2,AB2W04NI4OEAD,1881509818,J. Fernald,"[2, 2]",If you don't have a 3/32 punch or would like t...,4,"It's a punch, that's all.",1330387200,"02 28, 2012"
3,A148SVSWKTJKU6,1881509818,"Jusitn A. Watts ""Maverick9614""","[0, 0]",This works no better than any 3/32 punch you w...,4,It's a punch with a Glock logo.,1328400000,"02 5, 2012"
4,AAAWJ6LW9WMOO,1881509818,Material Man,"[0, 0]",I purchased this thinking maybe I need a speci...,4,"Ok,tool does what a regular punch does.",1366675200,"04 23, 2013"


In [9]:
df.shape

(296337, 9)

# Simple pre-preocessing and Tokenizing

1. Converting words to lower case.
2. Trimming spaces, removing punctuations.
3. Remove stop words.
4. Convert words to root forms like 'running' to 'run'.

To convert all these, gensim library is used.

In [13]:
df.reviewText.loc[0]

'This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy'

In [12]:
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [14]:
review_text.loc[0]

['this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 'very',
 'easy']

# Training the Word2Vec model

Train the model for reviews. Use a window of size 10 i.e. 10 words before the present word and 10 words ahead. A sentence with at least 2 words should only be considered, configure this using min_count parameter.

Workers define how many CPU threads to be used.

#### Model Initialization

In [15]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

#### Build Vocabulary

In [16]:
model.build_vocab(review_text, progress_per = 1000)

#### Train the Word2Vec model

In [17]:
model.corpus_count

296337

In [18]:
model.epochs

5

In [20]:
model.train(review_text, total_examples = model.corpus_count, epochs = model.epochs)

(91346447, 121496535)

#### Saving the model

In [21]:
model.save('sports-and-outdoors-reviews.model')

#Finding similar words and similarity between words

In [23]:
model.wv.most_similar("badminton")

[('volleyball', 0.8005030155181885),
 ('speedminton', 0.7967354655265808),
 ('soccer', 0.7781500816345215),
 ('tennis', 0.7573623657226562),
 ('basketball', 0.7565752863883972),
 ('softball', 0.7527838349342346),
 ('bocce', 0.7449088096618652),
 ('football', 0.7345447540283203),
 ('baseballs', 0.7288554906845093),
 ('lacrosse', 0.7244349718093872)]

In [27]:
model.wv.similarity(w1 = 'sports', w2 = 'games')

0.5761167